# **Project 1: SVM's for Binary and Multiclass Classification**

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import numpy as np
from numpy import linalg as LA
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw_ecoli = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ecoli.data', delimiter='\s+', header = None)

In [ ]:
features = list(range(1,8))
X = raw_ecoli.iloc[:,features]
y = raw_ecoli.iloc[:,8]

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
raw_ecoli['integer_label'] = y

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **SKLearn implementation of SVM with various kernels**

In [ ]:
mod_lin = svm.SVC(kernel='linear')
mod_lin.fit(X_train, y_train)
y_pred = mod_lin.predict(X_test)

accuracy = round(accuracy_score(y_test, y_pred),2)
conf_matrix = confusion_matrix(y_test, y_pred)

print('LINEAR SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

LINEAR SVM
----------
Accuracy: 0.91
Confusion Matrix:
[[29  0  0  0  0  0]
 [ 0 11  5  0  0  0]
 [ 0  1  6  0  0  0]
 [ 0  0  0  4  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0 11]]


In [ ]:
mod_poly = svm.SVC(kernel='poly', degree=2)
mod_poly.fit(X_train, y_train)
y_pred = mod_poly.predict(X_test)

accuracy = round(accuracy_score(y_test, y_pred),2)
conf_matrix = confusion_matrix(y_test, y_pred)

print('POLYNOMIAL SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

POLYNOMIAL SVM
----------
Accuracy: 0.74
Confusion Matrix:
[[28  1  0  0  0  0]
 [ 3  8  5  0  0  0]
 [ 1  2  4  0  0  0]
 [ 0  0  0  4  0  0]
 [ 0  0  0  0  1  0]
 [ 6  0  0  0  0  5]]


In [ ]:
mod_rbf = svm.SVC(kernel='rbf', gamma=1)
mod_rbf.fit(X_train, y_train)
y_pred = mod_rbf.predict(X_test)

accuracy = round(accuracy_score(y_test, y_pred),2)
conf_matrix = confusion_matrix(y_test, y_pred)

print('RADIAL BASIS SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

RADIAL BASIS SVM
----------
Accuracy: 0.93
Confusion Matrix:
[[29  0  0  0  0  0]
 [ 0 13  3  0  0  0]
 [ 0  1  6  0  0  0]
 [ 0  1  0  3  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0 11]]


In [ ]:
mod_sig = svm.SVC(kernel='sigmoid')
mod_sig.fit(X_train, y_train)
y_pred = mod_sig.predict(X_test)

accuracy = round(accuracy_score(y_test, y_pred),2)
conf_matrix = confusion_matrix(y_test, y_pred)

print('SIGMOID SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

SIGMOID SVM
----------
Accuracy: 0.91
Confusion Matrix:
[[29  0  0  0  0  0]
 [ 0 11  5  0  0  0]
 [ 0  1  6  0  0  0]
 [ 0  0  0  4  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0 11]]


# **Our implementation of SVM with various kernels**

In [ ]:
def binary_classifier(X, Y, kernel='linear', gamma=0.1, degree=2, r=1):
    N = len(Y)
    A = np.zeros([N, N]) #kernel matrix

    if kernel == 'linear':
        for i in range(N):
            for j in range(i, N):
                A[i, j] = np.sum(np.dot((X[i, :] * Y[i]).T, X[j, :] * Y[j]))
                A[j, i] = A[i, j]
    elif kernel == 'rbf':
        for i in range(N):
            for j in range(i, N):
                A[i, j] = np.exp(-gamma * LA.norm(X[i, :] - X[j, :]) ** 2)
                A[j, i] = A[i, j]
    elif kernel == 'poly':
        for i in range(N):
            for j in range(i, N):
                A[i, j] = (gamma * np.dot(X[i, :], X[j, :]) + r) ** degree
                A[j, i] = A[i, j]
    elif kernel == 'sigmoid':
        for i in range(N):
            for j in range(i, N):
                A[i, j] = np.tanh(gamma * np.sum(np.dot((X[i, :] * Y[i]).T, X[j, :] * Y[j])) + r)
                A[j, i] = A[i, j]

#intermediate weights for solving classification weights
    YM = np.outer(Y[1:], Y[1:])
    AY = np.outer(A[0, 1:], Y[1:])
    YA = np.outer(Y[1:], A[0, 1:])

    Y0S = Y[0] ** 2 #first label
    M = A[1:, 1:] + A[0, 0] * YM / Y0S - AY / Y[0] - YA / Y[0] #modified kernel matrix A

    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - Y / Y[0]
# solving for weight aw
    adiag = np.diagonal(A)
    t1 = A[0, 0]
    t2 = 2 * np.multiply(A[0, :], Y) / Y[0]
    t3 = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]] = np.divide(bright[indxnzero], t3[indxnzero])
    aw[0] = -sum(Y[indxnzero] * aw[indxnzero]) / Y[0]

    YA = np.zeros(N)
    YA[0] = Y[0] * aw[0]
    YA[indxnzero] = Y[indxnzero] * aw[indxnzero]

    wght = sum(X * YA[:, None])
    b = sum(Y - np.matmul(X, wght)) / N

    return (wght, b)

In [ ]:
def one_vs_all_classifier(X, Y, kernel='linear', gamma=1, degree=2, r=1):
    num_classes = len(np.unique(Y))
    classifiers = []

    for class_label in range(num_classes):
        Y_binary = np.where(Y == class_label, 1, -1)

        classifier = binary_classifier(X, Y_binary, kernel, gamma, degree, r)
        classifiers.append(classifier)

    return classifiers

In [ ]:
def predict_one_vs_all(X, classifiers):
    decision_scores = np.zeros((len(X), len(classifiers)))

    for i, (w, b) in enumerate(classifiers):
        decision_scores[:, i] = np.dot(X, w) + b

    predicted_labels = np.argmax(decision_scores, axis=1)

    return predicted_labels

In [ ]:
classifiers = one_vs_all_classifier(X_train, y_train, kernel='linear')
predicted_labels = predict_one_vs_all(X_test, classifiers)
accuracy = round(np.mean(predicted_labels == y_test),2)
conf_matrix = confusion_matrix(y_test, predicted_labels)

print('OUR LINEAR SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

OUR LINEAR SVM
----------
Accuracy: 0.75
Confusion Matrix:
[[28  0  0  1  0  0]
 [ 1 15  0  0  0  0]
 [ 0  7  0  0  0  0]
 [ 0  4  0  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  3  0  0  0  8]]


In [ ]:
classifiers = one_vs_all_classifier(X_train, y_train, kernel='poly', degree=2)
predicted_labels = predict_one_vs_all(X_test, classifiers)
accuracy = round(np.mean(predicted_labels == y_test),2)
conf_matrix = confusion_matrix(y_test, predicted_labels)

print('OUR POLYNOMIAL SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

OUR POLYNOMIAL SVM
----------
Accuracy: 0.71
Confusion Matrix:
[[29  0  0  0  0  0]
 [ 1 15  0  0  0  0]
 [ 0  7  0  0  0  0]
 [ 0  4  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  7  0  0  0  4]]


In [ ]:
classifiers = one_vs_all_classifier(X_train, y_train, kernel='rbf', gamma=1)
predicted_labels = predict_one_vs_all(X_test, classifiers)
accuracy = round(np.mean(predicted_labels == y_test),2)
conf_matrix = confusion_matrix(y_test, predicted_labels)

print('OUR RADIAL BASIS SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

OUR RADIAL BASIS SVM
----------
Accuracy: 0.75
Confusion Matrix:
[[28  0  0  1  0  0]
 [ 1 15  0  0  0  0]
 [ 0  7  0  0  0  0]
 [ 0  4  0  0  0  0]
 [ 0  0  0  0  1  0]
 [ 0  4  0  0  0  7]]


In [ ]:
classifiers = one_vs_all_classifier(X_train, y_train, kernel='sigmoid', gamma = 0.1, r = 1)
predicted_labels = predict_one_vs_all(X_test, classifiers)
accuracy = round(np.mean(predicted_labels == y_test),2)
conf_matrix = confusion_matrix(y_test, predicted_labels)

print('OUR SIGMOID SVM')
print('----------')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

OUR SIGMOID SVM
----------
Accuracy: 0.75
Confusion Matrix:
[[28  0  0  1  0  0]
 [ 1 15  0  0  0  0]
 [ 0  7  0  0  0  0]
 [ 0  4  0  0  0  0]
 [ 0  0  0  0  1  0]
 [ 0  4  0  0  0  7]]


In [ ]:
results = [["Linear", 0.91, 0.75],
           ["Polynomial", 0.74, 0.71],
           ["Radial Basis", 0.93, 0.75],
           ["Sigmoid", 0.91, 0.75]]

results_df = pd.DataFrame(results, columns=["Kernel","SKLearn","Ours"])
print(results_df.to_markdown(index=False))

| Kernel       |   SKLearn |   Ours |
|:-------------|----------:|-------:|
| Linear       |      0.91 |   0.75 |
| Polynomial   |      0.74 |   0.71 |
| Radial Basis |      0.93 |   0.75 |
| Sigmoid      |      0.91 |   0.75 |


# **Binary Classification Example**
Testing Cytoplasm (cp) against all

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

raw_ecoli = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ecoli.data', delimiter='\s+', header=None)

# Selecting features and labels
features_binary = list(range(1,8))
X = raw_ecoli.iloc[:, features_binary]
y = raw_ecoli.iloc[:, -1]  # Labels

#Search for cytoplasm, 1 if cp, -1 if not
binary_y = np.where(raw_ecoli.iloc[:, -1] == 'cp', 1, -1)

X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(X, binary_y, test_size=0.2, random_state=42)

X_train_binary_np = X_train_binary.to_numpy()
X_test_binary_np = X_test_binary.to_numpy()


In [ ]:
# X = Feature matrix (for future pred. data)
def predict(X, w, b):
    # Prediction function
    decision_values = np.dot(X, w) + b
    predictions = np.sign(decision_values)
    return predictions

# Kernels to test
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

# Iterate over kernels and compare SVMs
for kernel in kernels:
    # Train and evaluate custom SVM
    w, b = binary_classifier(X_train_binary_np, y_train_binary, kernel=kernel, gamma=0.1, degree=3, r=1)
    y_pred_custom = predict(X_test_binary_np, w, b)
    accuracy_custom = accuracy_score(y_test_binary, y_pred_custom)
    print(f"Custom SVM Kernel: {kernel}, Accuracy: {accuracy_custom}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_binary, y_pred_custom))

    # Train and evaluate SKLearn's SVM
    model = SVC(kernel=kernel, gamma=0.1, degree=3)
    model.fit(X_train_binary_np, y_train_binary)
    y_pred_sklearn = model.predict(X_test_binary_np)
    accuracy_sklearn = accuracy_score(y_test_binary, y_pred_sklearn)
    print(f"SKLearn SVM Kernel: {kernel}, Accuracy: {accuracy_sklearn}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_binary, y_pred_sklearn))
    print("\n")

Custom SVM Kernel: linear, Accuracy: 0.8970588235294118
Confusion Matrix:
[[30  6]
 [ 1 31]]
SKLearn SVM Kernel: linear, Accuracy: 0.9411764705882353
Confusion Matrix:
[[34  2]
 [ 2 30]]


Custom SVM Kernel: poly, Accuracy: 0.8529411764705882
Confusion Matrix:
[[29  7]
 [ 3 29]]
SKLearn SVM Kernel: poly, Accuracy: 0.5294117647058824
Confusion Matrix:
[[36  0]
 [32  0]]


Custom SVM Kernel: rbf, Accuracy: 0.8676470588235294
Confusion Matrix:
[[29  7]
 [ 2 30]]
SKLearn SVM Kernel: rbf, Accuracy: 0.9411764705882353
Confusion Matrix:
[[34  2]
 [ 2 30]]


Custom SVM Kernel: sigmoid, Accuracy: 0.8823529411764706
Confusion Matrix:
[[29  7]
 [ 1 31]]
SKLearn SVM Kernel: sigmoid, Accuracy: 0.9411764705882353
Confusion Matrix:
[[34  2]
 [ 2 30]]




In [ ]:
results_binary = [["Linear", 0.94, 0.89],
           ["Polynomial", 0.53, 0.85],
           ["Radial Basis", 0.94, 0.87],
           ["Sigmoid", 0.94, 0.88]]

results_binary_df = pd.DataFrame(results_binary, columns=["Kernel","SKLearn","Ours"])
print(results_binary_df.to_markdown(index=False))

| Kernel       |   SKLearn |   Ours |
|:-------------|----------:|-------:|
| Linear       |      0.94 |   0.89 |
| Polynomial   |      0.53 |   0.85 |
| Radial Basis |      0.94 |   0.87 |
| Sigmoid      |      0.94 |   0.88 |
